# Image Downloader via Roboflow

download image has been bounded box help by roboflow

In [ ]:
!pip install roboflow

In [ ]:
ROBOFLOW_API="4V6T95yomp7w4ccYHkYr"

In [ ]:
import os
import json

from roboflow import Roboflow
rf  = Roboflow(api_key=ROBOFLOW_API)
project = rf.workspace("foreveyrone").project("roadcrack-4xqqm")

loading Roboflow workspace...
loading Roboflow project...


In [ ]:
version = project.version(1)

In [ ]:
dataset = version.download("coco")

In [ ]:
import os
with open("/content/RoadCrack-1/train/_annotations.coco.json","r") as f:
  obj = json.loads(f.read())

In [ ]:
obj

{'info': {'year': '2025',
  'version': '1',
  'description': 'Exported from roboflow.com',
  'contributor': '',
  'url': 'https://public.roboflow.com/object-detection/undefined',
  'date_created': '2025-11-14T17:02:13+00:00'},
 'licenses': [{'id': 1,
   'url': 'https://creativecommons.org/licenses/by/4.0/',
   'name': 'CC BY 4.0'}],
 'categories': [{'id': 0, 'name': 'objects', 'supercategory': 'none'},
  {'id': 1, 'name': 'retak', 'supercategory': 'objects'}],
 'images': [{'id': 0,
   'license': 1,
   'file_name': 'IMG_20251106_150718_jpg.rf.d00bb8c367d8802fc9783c54cd9f470a.jpg',
   'height': 2448,
   'width': 2448,
   'date_captured': '2025-11-14T17:02:13+00:00',
   'extra': {'name': 'IMG_20251106_150718.jpg'}},
  {'id': 1,
   'license': 1,
   'file_name': 'IMG_20251106_150738_1_jpg.rf.1c0ec0d7f3fee92338fb1c37be1638f2.jpg',
   'height': 2448,
   'width': 2448,
   'date_captured': '2025-11-14T17:02:13+00:00',
   'extra': {'name': 'IMG_20251106_150738_1.jpg'}},
  {'id': 2,
   'license':